# Data Cleaning and Machine Learning 

## Reading the New Integrated Sample

In [1]:
import numpy as np
import imdb 
import pandas as pd
anime_data = pd.read_csv("/Users/sezin/PycharmProjects/dsa-DC4ML/new_anime_data.csv")

anime_data['episodes'] = anime_data['episodes'].replace('Unknown', np.nan)


## Data Transformation

### Drop Some Special Characters from "genre" and "overview", and Covert "type" to Movie/Tv series

In [2]:
l1 = []
l2 = []
l3 = []


anime_data['genre'].fillna('',  inplace=True)
anime_data['overview'].fillna('',  inplace=True)
anime_data['type'].fillna('',  inplace=True)
for index, row in anime_data.iterrows():
    item = row['genre']
    if(pd.isnull(item)):
            item =""
    else:
        if isinstance(item, (list, tuple)):
            item = ','.join(item)
        else:
            item = item.replace(" ","")
            item = item.replace("[","")
            item = item.replace("]","")
            item = item.replace("'","")
    l1.append(item) 
      

for index, row in anime_data.iterrows():
    item = row['overview']
    if(pd.isnull(row['overview'])):
            item = ""
    else:
       # for item in anime_data['overview']:
            if isinstance(item, (list, tuple)):
                item = ','.join(item)
            else:
                item = item.replace("[","")
                item = item.replace("]","")
    l2.append(item) 

for index, row in anime_data.iterrows():
    item = row['type']
    if(pd.isnull(row['type'])):
        item = np.nan
    else:    
        if "movie" in item:
            item = "movie"
        else:
            item = "tv series"
    l3.append(item)   
    
    
anime_data['genre'] = l1   
anime_data['overview'] = l2
anime_data['type'] = l3

#drop dublicate
#anime_data.drop_duplicates(inplace = True)

### Applying One-Hot and Binary Encoding

In [3]:
from sklearn.preprocessing import LabelBinarizer

anime_data = anime_data.dropna()

#one-hot encoding to transform the genres column to numerical columns
df = anime_data.genre.str.get_dummies(',')

#binary encoding for type column
type_lb = LabelBinarizer()
X = type_lb.fit_transform(anime_data.type.values)


dfOneHot = pd.DataFrame(X, columns = ["movie/TVseries" for i in range(X.shape[1])])
anime_data = pd.concat([anime_data, dfOneHot], axis=1)
anime_data = pd.concat([anime_data, df], axis=1)

anime_data['movie/TVseries'].fillna(0, inplace=True)

#anime_data

### Counting words "overview"

In [4]:
import nltk
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
def get_words(x):
    bagofwords=[]
    for i in x:
        if i[1]=='NN':
            bagofwords.append(i[0])
        elif i[1]=='NNS':
            bagofwords.append(i[0])
        elif i[1]=='NNP':
            bagofwords.append(i[0])
        elif i[1]=='NNPS':
            bagofwords.append(i[0])
        elif i[1]=='JJ':
            bagofwords.append(i[0])
        elif i[1]=='JJR':
            bagofwords.append(i[0])
        elif i[1]=='JJS':
            bagofwords.append(i[0])
        elif i[1]=='RB':
            bagofwords.append(i[0])
        elif i[1]=='RBR':
            bagofwords.append(i[0])
        elif i[1]=='RBS':
            bagofwords.append(i[0])
    return bagofwords

def clean_words(x):
    b=nltk.pos_tag(nltk.word_tokenize(x))
    result=get_words(b)
    return result

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

summary_doc = anime_data['overview'].fillna("").map(clean_words)
summary_doc =summary_doc.apply(','.join)
 
vectorizer = TfidfVectorizer()
overview_feature = vectorizer.fit_transform(summary_doc).toarray()
#overview_feature = vectorizer.fit_transform(summary_doc)


df = pd.DataFrame(overview_feature, columns = ["word"+ str(int(i)) for i in range(overview_feature.shape[1])])
anime_data = pd.concat([anime_data, df], axis=1)

#drop Null values
#anime_data = anime_data.dropna(inplace=True)

In [6]:
'''
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
overview_feature = vectorizer.fit_transform(anime_data['overview']).toarray()

df = pd.DataFrame(overview_feature, columns = ["word"+ str(int(i)) for i in range(overview_feature.shape[1])])
anime_data = pd.concat([anime_data, df], axis=1)
'''

'\nfrom sklearn.feature_extraction.text import CountVectorizer\n\nvectorizer = CountVectorizer()\noverview_feature = vectorizer.fit_transform(anime_data[\'overview\']).toarray()\n\ndf = pd.DataFrame(overview_feature, columns = ["word"+ str(int(i)) for i in range(overview_feature.shape[1])])\nanime_data = pd.concat([anime_data, df], axis=1)\n'

## Feature Extraction

In [7]:
anime_data = anime_data.drop(columns=['Unnamed: 0', 'anime_id', 'name', 'genre', 'overview', 'type'])


### Drop None Values

In [8]:


anime_data = anime_data.dropna()
print(anime_data.shape)


(88, 1854)


## Modeling

In [9]:
from sklearn import preprocessing

y = anime_data['rating']
X = anime_data.drop(columns=['rating'])
anime_data.shape

(88, 1854)

## Selecting Best Features

### 1) Filter Method - Univariate Selection

In [10]:
'''from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

selector = SelectKBest(score_func=f_regression, k=1853)
fit = selector.fit(X, y)
# summarize scores
np.set_printoptions(precision=3)
#print(fit.scores_)
features = fit.transform(X)
print(features.shape)
print(X.shape)'''

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

selector = SelectKBest(score_func=f_regression,k=700)#anime_X_test.shape[1]-1)
features = selector.fit(X, y)

# summarize scores
np.set_printoptions(precision=3)
#print(fit.scores_)

print(X.shape)
X = features.transform(X)
print(X.shape)


(88, 1853)
(88, 700)


## Spliting the Data

In [15]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split

X = anime_data.drop(columns=['rating'])
y = anime_data['rating']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Split the data into training/testing sets
#anime_X_train = features[:-500]
#anime_X_test = features[-500:]

# Split the targets into training/testing sets
#anime_y_train = y[:-500]
#anime_y_test = y[-500:]


## Training and Testing

In [16]:
# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train,y_train)

# Make predictions using the testing set
anime_y_pred = regr.predict(X_test)

## Evaluation

In [18]:
# The mean absolute error
print("Mean absolute error: %.2f" % np.sqrt(mean_absolute_error(y_test, anime_y_pred)))

# The mean squared error
print("Mean squared error: %.2f" % np.sqrt(mean_squared_error(y_test, anime_y_pred)))

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test,anime_y_pred))



Mean absolute error: 0.52
Mean squared error: 0.33
Variance score: -1.07
